In [ ]:
import scipy.ndimage as sci
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
import tensorflow as tf
from tensorflow import keras


In [ ]:
#Load Data and Pre-processing and Visualization
def load_dataSet():
    data = ['forward', 'left', 'right', 'idle']
    X = []
    sizes = []
    for direction in data:
        address = './' + direction + '/*.jpg'
        file_list = glob.glob(address)
        sizes.append(len(file_list))
        print('len of {} is {}'.format(direction, len(file_list)))
        for image in file_list:
            x = Image.open(image)
            x = np.array(x.convert('L'))
            x = x / np.max(x)
            X.append(x)
    #lets define 0 as left, 1 as forward and 2 as right
    y_forward = 1* np.ones((sizes[0],1))
    y_left = np.zeros((sizes[1], 1))
    y_right = 2*np.ones((sizes[2],1))
    y_idle = 3*np.ones((sizes[3], 1))
    Y = np.concatenate((y_forward, y_left, y_right, y_idle),axis=0)
    sequence = np.arange(sum(sizes))
    np.random.shuffle(sequence)
    
    X = np.array(X)
    X = X[sequence, :]
    Y = Y[sequence, :]
    
    return X, Y

In [ ]:
X, Y = load_dataSet()

In [ ]:
j = 1;
plt.figure(figsize=(10,10))
for i in np.random.randint(0, Y.shape[0], 9):
    plt.subplot(3, 3, j)
    plt.imshow(X[i],cmap="gray")
    plt.grid(False)
    plt.xlabel('{}'.format(Y[i]))
    j += 1
plt.show()

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape= (X.shape[1], X.shape[2])),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.softmax)
])

model.compile(optimizer='sgd',
             loss='mean_squared_error',
             metrics=['accuracy'])

model.fit(X, Y, epochs=2)

In [ ]:
data = ['forward', 'left', 'right', 'idle']
X_test = []
test_sizes = []
for direction in data:
    address = './' + direction + '_test/*.jpg'
    file_list = glob.glob(address)
    test_sizes.append(len(file_list))
    print('len of {} is {}'.format(direction, len(file_list)))
    for image in file_list:
        x = Image.open(image)
        x = np.array(x.convert('L'))
        X_test.append(x)
X_test = np.array(X_test)
y_forward = 1* np.ones((test_sizes[0],1))
y_left = np.zeros((test_sizes[1], 1))
y_right = 2*np.ones((test_sizes[2],1))
y_idle = 3*np.ones((test_sizes[3], 1))
Y_test = np.concatenate((y_forward, y_left, y_right, y_idle),axis=0)
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)